In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing Necessary Libraries

In [2]:
import os
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Define Custom Dataset Class


In [3]:
class CreateDataset(Dataset):
    def __init__(self, df_data, data_dir='../input/', transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_name, label = self.df[index]
        img_path = os.path.join(self.data_dir, img_name + '.png')
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

Image transformations

In [4]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.4),
    transforms.RandomRotation(15),  
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), 
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

Load the training Data


In [5]:
train_path = "/kaggle/input/aptos2019-blindness-detection/train_images"
train_csv = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/train.csv')
train_data = CreateDataset(df_data=train_csv, data_dir=train_path, transform=train_transforms)

Split Training Data into Training and Validation Data

In [6]:
train_indices, valid_indices = train_test_split(np.arange(len(train_data)), test_size=0.2, random_state=42)
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)

Data Loaders

In [7]:
from torch.utils.data import WeightedRandomSampler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# class_counts = train_csv['diagnosis'].value_counts().sort_index().values
# class_weights = 1.0 / torch.tensor(class_counts, dtype=torch.float).to(device)

# sample_weights = class_weights[train_csv['diagnosis'].values]
# sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

trainloader = DataLoader(train_data, batch_size=32, sampler= train_sampler, num_workers=4)
validloader = DataLoader(train_data, batch_size=32, sampler=valid_sampler, num_workers=4)

Model Setup

In [9]:
model = models.efficientnet_b3(pretrained=True) 
num_ftrs = model.classifier[1].in_features 
out_ftrs = 5
model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Linear(512, out_ftrs),
    nn.LogSoftmax(dim=1)
)
model.to(device)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

Loss and optimizer

In [10]:

criterion = nn.NLLLoss()

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.00001, weight_decay=1e-4)

Function to Train and Validate Model

In [11]:
def train_and_validate(epochs, freeze_until = 3, unfreeze_step = 4):
    valid_loss_min = np.Inf
    train_losses, valid_losses, acc = [], [], []
    
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the final layer initially for training
    for param in model.classifier.parameters():
        param.requires_grad = True

    unfreeze_counter = 0
    
    for epoch in range(epochs):
        if epoch >= freeze_until and unfreeze_counter < len(list(model.children())):
            
            layers_to_unfreeze = list(model.children())[unfreeze_counter:unfreeze_counter + unfreeze_step]
            for layer in layers_to_unfreeze:
                for param in layer.parameters():
                    param.requires_grad = True
            print(f"Unfreezing layers from index {unfreeze_counter} to {unfreeze_counter + unfreeze_step - 1} at epoch {epoch + 1}.")
            unfreeze_counter += unfreeze_step
            
        model.train()
        running_loss = 0
        
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation
        model.eval()
        test_loss = 0
        accuracy = 0
        with torch.no_grad():
            for images, labels in validloader:
                images, labels = images.to(device), labels.to(device)
                logps = model(images)
                test_loss += criterion(logps, labels).item()
                ps = torch.exp(logps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
        # Average losses and accuracies
        train_loss = running_loss / len(trainloader)
        valid_loss = test_loss / len(validloader)
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        acc.append(accuracy / len(validloader))
        print(f"Epoch: {epoch+1}/{epochs}.. "
              f"Training Loss: {train_loss:.3f}.. "
              f"Validation Loss: {valid_loss:.3f}.. "
              f"Validation Accuracy: {accuracy / len(validloader):.3f}")

        # Save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}).  Saving model ...')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': valid_loss_min
            }, 'best_model.pth')
            valid_loss_min = valid_loss

print('Training Completed Successfully!')


Training Completed Successfully!


Execute

In [13]:
train_and_validate(epochs=20, freeze_until = 2, unfreeze_step = 2)

Epoch: 1/20.. Training Loss: 1.391.. Validation Loss: 1.334.. Validation Accuracy: 0.505
Validation loss decreased (inf --> 1.334299).  Saving model ...
Epoch: 2/20.. Training Loss: 1.249.. Validation Loss: 1.229.. Validation Accuracy: 0.553
Validation loss decreased (1.334299 --> 1.229269).  Saving model ...
Unfreezing layers from index 0 to 1 at epoch 3.
Epoch: 3/20.. Training Loss: 1.082.. Validation Loss: 0.999.. Validation Accuracy: 0.656
Validation loss decreased (1.229269 --> 0.999302).  Saving model ...
Unfreezing layers from index 2 to 3 at epoch 4.
Epoch: 4/20.. Training Loss: 0.921.. Validation Loss: 0.880.. Validation Accuracy: 0.719
Validation loss decreased (0.999302 --> 0.880476).  Saving model ...
Epoch: 5/20.. Training Loss: 0.831.. Validation Loss: 0.817.. Validation Accuracy: 0.733
Validation loss decreased (0.880476 --> 0.817135).  Saving model ...
Epoch: 6/20.. Training Loss: 0.780.. Validation Loss: 0.768.. Validation Accuracy: 0.738
Validation loss decreased (0.8